https://radimrehurek.com/gensim/models/keyedvectors.html

In [1]:
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from pymystem3 import Mystem
from multiprocessing import Pool
import time

In [2]:
word_vectors = KeyedVectors.load_word2vec_format('/data/gensim/news_0_300_2.bin.gz', binary=True)
word_vectors.init_sims(replace=True)

In [3]:
mystem = Mystem()

In [4]:
mystem_part = {
    'A': 'ADJ', # прилагательное
    'ADV': 'ADV', # наречие
    'ADVPRO': 'ADV', # местоименное наречие
    'ANUM': 'ADJ', # числительное-прилагательное
    'APRO': 'DET', # местоимение-прилагательное
    'COM': 'ADJ', # часть композита - сложного слова
    'CONJ': 'SCONJ', # союз
    'INTJ': 'INTJ',	# междометие
    'NUM': 'NUM', # числительное
    'PART': 'PART',	# частица
    'PR': 'ADP', # предлог
    'S': 'NOUN', # существительное
    'SPRO': 'PRON',
    'V': 'VERB', # глагол
}

In [5]:
def lemm(word):
    a = mystem.analyze(word)
    
    if len(a) == 0:
        return word
    
    analysis = a[0]['analysis']
    
    if len(analysis) == 0:
        return word
    
    gr = analysis[0]
    parts = gr['gr'].split("=")
    
    if len(parts) == 0:
        return word
    
    parts2 = parts[0].split(",")
        
    if len(parts2) == 0:
        return word
    
    part = parts2[0]
        
    if part in mystem_part:
        return "%s_%s" % (analysis[0]['lex'], mystem_part[part])
    else:
        return word

In [36]:
def similar(word, lemming=True):
    try:
        if lemming:
            w = lemm(word)
        else:
            w = word
        sim = word_vectors.most_similar(positive=[w])
        return list(map(lambda x: x[0], sim))
    except Exception as e:
        return [word]

In [26]:
def extractSemanticGroup(seq, max_inter = 1):
    last_res = map(lambda x: map(lambda y: lemm(y), x.split(" ")), seq)
    for i in range(0, max_inter):
        sem_groups = {}
        res_seq = []
        for words in last_res:
            res_tokens = []
            for token in words:
                if token not in sem_groups:
                    sims = similar(token, False)
                    sem_groups[token] = token
                    res_tokens.append(token)
                    for sim in sims:
                       sem_groups[sim] = token
                else:
                    res_tokens.append(sem_groups[token])
            res_seq.append(res_tokens)
        last_res = res_seq
    result = map(lambda x: " ".join(map(lambda y: y.split("_")[0], x)), last_res)
    return list(result)

In [27]:
def unic_count(seq):
    total = []
    for s in seq:
        total.extend(s.split(" "))
    return len(set(total))

In [12]:
news = [
    "свидетель редкий природный явление становиться житель москва первый день февраль рано утро многий разбудить раскат гром настоящий зимний гроза сопровождаться молния порыв ветер метр секунда ледяной дождь новый месяц",
    "обвинять государственный измена домохозяйка вязьма светлана давыдов освобождать подписка невыезд страница фейсбук сообщать адвокат женщина иван павлов ходатайство защита изменение мера пресечение удовлетворять следователь взять подписка невыезд мать смочь вернуться ребенок сообщать защитник давыдова судя адвокат новость",
    "пока ребенок школа выращивать фасоль выращивать грудь говорить героиня популярный сериал глупо звучать грудь выращивать подращивать немного итак нужно делать качать мышца грудь поддерживать грудной мышца любой мышца тело подкачивать укреплять интернет смочь узнавать упражнение тренажер дом коврик помогать прорабатывать мышца грудь мышца окрепнуть подрастать грудь выглядеть пить молоко бабушка советовать капуста грудь расти"
]

In [33]:
start = time.time()
res = extractSemanticGroup(news, 5)
print(res)
print("time: %s s" % (time.time() - start))

"word 'пока_ADV' not in vocabulary"
"word 'подращивать_VERB' not in vocabulary"
"word 'итак_SCONJ' not in vocabulary"
"word 'пока_ADV' not in vocabulary"
"word 'подращивать_VERB' not in vocabulary"
"word 'итак_SCONJ' not in vocabulary"
"word 'пока_ADV' not in vocabulary"
"word 'подращивать_VERB' not in vocabulary"
"word 'итак_SCONJ' not in vocabulary"
"word 'пока_ADV' not in vocabulary"
"word 'подращивать_VERB' not in vocabulary"
"word 'итак_SCONJ' not in vocabulary"
"word 'пока_ADV' not in vocabulary"
"word 'подращивать_VERB' not in vocabulary"
"word 'итак_SCONJ' not in vocabulary"
['свидетель редкий природный явление становиться житель москва первый день февраль рано утро многий разбудить раскат гром настоящий зимний гроза сопровождаться молния порыв порыв метр секунда ледяной порыв новый день', 'обвинять государственный измена домохозяйка вязьма светлана давыдов освобождать подписка подписка страница страница сообщать адвокат женщина иван павлов ходатайство защита изменение мера пре

In [34]:
unic_count(res)

96

In [35]:
unic_count(news)

103